TODO: downweight zero sales and use frequency on test store-item pair to train

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from pandas.tseries.offsets import MonthEnd

In [2]:
def isConsecutive(seq):
    # if non-consecutive, need to re-map to consecutive number starting from 1
    uniq = pd.unique(seq)
    return len(uniq) == (uniq.max()-uniq.min() + 1)

In [3]:
def isUniqBigger(seq1,seq2):
    return set(seq1) >= set(seq2)

In [4]:
def reMapDF(df,cols):
    # remap cols in dataframe to consecutive integers starting from one
    for col in cols:
        uniq = pd.unique(df[col])
        dict_ = {item:i+1 for i,item in enumerate(uniq)}
        df = df.replace({col:dict_})
    return df

In [5]:
def dimentionDF(df,cols):
    return {col:len(set(df[col])) for col in cols}

In [6]:
def mergeFillCast(df1,df2,key):
    cols = df2.columns.values
    types = df2.dtypes.values
    dict_ = {col:type_ for col,type_ in zip(cols,types)}
    dfOut = pd.merge(df1, df2, how='left', on=key, 
             suffixes=('', '_y'), copy=True, indicator=False).fillna(0)
    dfOut[cols] = \
        dfOut[cols].astype(dict_)
    return dfOut

In [7]:
def mergeFillCastsss(df0,dfs,keys):
    for df,key in zip(dfs,keys):
        df0 = mergeFillCast(df0,df,key)
    return df0

In [8]:
types = {'id': 'int32',
         'item_nbr': 'int32',
         'store_nbr': 'int8',
         'unit_sales': 'float32',
         'onpromotion': bool}

In [9]:
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)

In [11]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,2
1,2013-01-01,25,105574,1.0,2
2,2013-01-01,25,105575,2.0,2
3,2013-01-01,25,108079,1.0,2
4,2013-01-01,25,108701,1.0,2


In [50]:
test = pd.read_csv('test.csv',parse_dates=['date'],dtype=types, infer_datetime_format=True)
test = test.fillna(2,axis=1)
test.onpromotion = test.onpromotion.astype(np.int8)

Needs to map items2 before mapping item_nbr

In [12]:
items = pd.read_csv('items.csv')
stores = pd.read_csv('stores.csv')

In [13]:
items2 = reMapDF(items,['family','class'])
items2[['family','class','perishable']] = \
        items2[['family','class','perishable']].astype('int16')

In [14]:
stores2 = reMapDF(stores,['city', 'state', 'type'])
stores2 = stores2.astype('int8')

In [15]:
val = train[train.date >= '2017-07-31']

In [16]:
train = train[train.date < '2017-07-31']

In [17]:
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}
iter_mapping = lambda x: item_dict[x] if x in item_dict else 0

Use the first date that store has sales on any items as the first date to densify the time series data.

In [8]:
def CreateData(data):
    SI_timeMinMax = data.groupby(['store_nbr','item_nbr'])['date'].agg([np.count_nonzero]).reset_index()
    SI_train_sales = data.groupby(['store_nbr','item_nbr'])[['date','unit_sales']].\
                    agg(lambda x: tuple(x)).reset_index()
    storeTime = data.groupby(['store_nbr'])['date'].agg([np.min,np.max]).reset_index()
    dfs = [items2,stores2,SI_train_sales,storeTime]
    keys = ['item_nbr','store_nbr',['item_nbr','store_nbr'],'store_nbr']
    SI_train = mergeFillCastsss(SI_timeMinMax,dfs,keys)
    SI_train['item_nbr'] = SI_train.item_nbr.map(iter_mapping)
    SI_train['sampleWeight'] = SI_train.count_nonzero/((SI_train.amax - SI_train.amin).dt.days)
    SI_train.drop('count_nonzero',1,inplace=True)
    return SI_train

In [47]:
trainRNN = CreateData(train)
valRNN = CreateData(val)

In [27]:
trainRNN.to_csv('trainRNN.csv',index=False)

In [30]:
valRNN.to_csv('valRNN.csv',index=False)

In [8]:
trainRNN = pd.read_csv('trainRNN.csv',engine='python')

In [17]:
valRNN = pd.read_csv('valRNN.csv',engine='python')

In [ ]:
def RNN_generator(data,batchSize,seqSize,shuffle=True):
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    n = data.shape[0]
    from_ = 0
    for i in range(n//batchSize):
        X = data.loc[from_:from_+batchSize,'store_nbr':'cluster'].values.astype(np.int32)
        
        
        from_ += batchSize

In [39]:
valRNN = valRNN.sample(frac=1).reset_index(drop=True)

In [45]:
valRNN

,store_nbr,item_nbr,family,class,perishable,city,state,type,cluster,date,unit_sales,amin,amax
0,34,2634,1,22,0,10,9,2,6,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(13.0, 4.0, 5.0, 16.0, 28.0, 1.0, 5.0, 20.0, 8...",2017-07-31,2017-08-15
1,35,1441,13,59,1,15,9,3,3,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(11.042, 15.366, 8.96, 11.691, 7.8130002, 8.12...",2017-07-31,2017-08-15
2,49,16,1,16,0,1,1,5,11,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(3.0, 13.0, 6.0, 3.0, 11.0, 3.0, 10.0, 5.0, 4....",2017-07-31,2017-08-15
3,24,1188,1,21,0,10,9,1,1,"(Timestamp('2017-08-02 00:00:00'), Timestamp('...","(3.0, 3.0, 4.0, 2.0, 2.0, 2.0, 4.0, 10.0, 5.0,...",2017-07-31,2017-08-15
4,2,3106,1,65,0,1,1,1,13,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(1.0, 1.0, 2.0, 3.0, 1.0)",2017-07-31,2017-08-15
5,50,3116,9,56,0,9,8,5,14,"(Timestamp('2017-08-02 00:00:00'), Timestamp('...","(1.0, 1.0, 1.0, 3.0, 2.0, 1.0, 1.0)",2017-07-31,2017-08-15
6,29,469,9,56,0,10,9,4,10,"(Timestamp('2017-08-05 00:00:00'),)","(1.0,)",2017-07-31,2017-08-15
7,49,335,2,47,0,1,1,5,11,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(4.0, 1.0, 9.0, 4.0, 5.0, 2.0, 1.0, 5.0, 1.0, ...",2017-07-31,2017-08-15
8,52,1900,7,192,0,21,16,5,11,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, ...",2017-07-31,2017-08-15
9,50,895,2,36,0,9,8,5,14,"(Timestamp('2017-07-31 00:00:00'), Timestamp('...","(5.0, 4.0, 5.0, 4.0, 7.0, 5.0, 1.0, 1.0, 1.0, ...",2017-07-31,2017-08-15


In [46]:
valRNN.loc[10:20,'store_nbr':'cluster'].values.astype(np.int32)

array([[  18, 1813,    7,  188,    0,    1,    1,    2,   16],
       [  41, 2467,   22,  271,    1,   19,   14,    1,    4],
       [  40,  749,    2,   95,    0,   19,   14,    3,    3],
       [  49, 3637,    2,  141,    0,    1,    1,    5,   11],
       [   4,    9,    1,   21,    0,    1,    1,    1,    9],
       [  51, 1637,   11,  181,    1,   10,    9,    5,   17],
       [  22,  860,    1,   10,    0,    8,    7,    3,    7],
       [  15, 3426,    4,  210,    1,    6,    5,    3,   15],
       [  29,  760,    1,    8,    0,   10,    9,    4,   10],
       [  18, 2440,   22,  282,    1,    1,    1,    2,   16],
       [  16, 2966,    1,  150,    0,    2,    2,    3,    3]], dtype=int32)

In [154]:
a = SI_train_sales.head(10)

In [163]:
pd.concat([a.apply(lambda x: pd.Series(x.date),axis=1).stack(),\
          a.apply(lambda x: pd.Series(x.unit_sales),axis=1).stack()],1)

0     1
0 0  2013-01-10   1.0
  1  2013-01-11   1.0
  2  2013-01-14   1.0
  3  2013-01-18   2.0
  4  2013-01-21   1.0
  5  2013-01-25   1.0
1 0  2013-01-04   2.0
  1  2013-01-05   3.0
  2  2013-01-07   2.0
  3  2013-01-08   6.0
  4  2013-01-09   3.0
  5  2013-01-10   1.0
  6  2013-01-11   2.0
  7  2013-01-12   1.0
  8  2013-01-15   1.0
  9  2013-01-16   3.0
  10 2013-01-17   3.0
  11 2013-01-18   1.0
  12 2013-01-20   2.0
  13 2013-01-21   3.0
  14 2013-01-22   5.0
  15 2013-01-23   5.0
  16 2013-01-24   2.0
  17 2013-01-25   3.0
  18 2013-01-26   4.0
2 0  2013-01-02   2.0
  1  2013-01-03   3.0
  2  2013-01-04   2.0
  3  2013-01-05   4.0
  4  2013-01-06   2.0
...         ...   ...
8 14 2013-01-17   9.0
  15 2013-01-18   5.0
  16 2013-01-19   9.0
  17 2013-01-20   4.0
  18 2013-01-21   7.0
  19 2013-01-22  10.0
  20 2013-01-23   1.0
  21 2013-01-24   1.0
  22 2013-01-25   6.0
  23 2013-01-26   7.0
9 0  2013-01-02   2.0
  1  2013-01-05   1.0
  2  2013-01-06   1.0
  3  2013-01-07   3.0
  4  2013-01-08   1.0
  5  2013-01-09   2.0
  6  2013-01-10   1.0
  7  2013-01-11   3.0
  8  2013-01-12   3.0
  9  2013-01-14   4.0
  10 2013-01-15   4.0
  11 2013-01-16   3.0
  12 2013-01-18   2.0
  13 2013-01-19   3.0
  14 2013-01-20   1.0
  15 2013-01-22   3.0
  16 2013-01-23   2.0
  17 2013-01-24   3.0
  18 2013-01-25   1.0
  19 2013-01-26   2.0

[192 rows x 2 columns]

In [168]:
SI_timeMinMax.head()

,store_nbr,item_nbr,amin,amax,count_nonzero
0,1,96995,2013-01-10,2017-07-22,181
1,1,99197,2014-08-20,2017-07-28,182
2,1,103520,2013-01-04,2017-07-29,1109
3,1,103665,2013-01-02,2017-07-30,1345
4,1,105574,2013-01-02,2017-07-30,1532


In [170]:
SI_timeMinMax.head(1000).apply(lambda x:pd.Series(pd.date_range(x.amin,periods=16)),axis=1).stack()

0    0    2013-01-10
     1    2013-01-11
     2    2013-01-12
     3    2013-01-13
     4    2013-01-14
     5    2013-01-15
     6    2013-01-16
     7    2013-01-17
     8    2013-01-18
     9    2013-01-19
     10   2013-01-20
     11   2013-01-21
     12   2013-01-22
     13   2013-01-23
     14   2013-01-24
     15   2013-01-25
1    0    2014-08-20
     1    2014-08-21
     2    2014-08-22
     3    2014-08-23
     4    2014-08-24
     5    2014-08-25
     6    2014-08-26
     7    2014-08-27
     8    2014-08-28
     9    2014-08-29
     10   2014-08-30
     11   2014-08-31
     12   2014-09-01
     13   2014-09-02
             ...    
998  2    2013-01-05
     3    2013-01-06
     4    2013-01-07
     5    2013-01-08
     6    2013-01-09
     7    2013-01-10
     8    2013-01-11
     9    2013-01-12
     10   2013-01-13
     11   2013-01-14
     12   2013-01-15
     13   2013-01-16
     14   2013-01-17
     15   2013-01-18
999  0    2013-01-02
     1    2013-01-03
     2    201

** Data Processing **

In [12]:
holidays_events = pd.read_csv('holidays_events.csv',parse_dates=['date'],infer_datetime_format=True)

In [13]:
holidays_events2 = reMapDF(holidays_events.drop('description',1),['type', 'locale', 'locale_name'])

In [14]:
holidays_events2[['type', 'locale', 'locale_name','transferred']] = \
        holidays_events2[['type', 'locale', 'locale_name','transferred']].astype('int8')

In [15]:
items = pd.read_csv('items.csv')

In [16]:
items2 = reMapDF(items,['family','class'])

In [17]:
items2[['family','class','perishable']] = \
        items2[['family','class','perishable']].astype('int8')

In [18]:
stores = pd.read_csv('stores.csv')

In [21]:
dateVar = pd.DataFrame(pd.date_range('2013-01-01', '2017-08-31'),columns=['date'])

In [22]:
dateVar['dayOfWeek'] = dateVar.date.dt.dayofweek

In [23]:
dateVar['payDay'] = ((dateVar.date.dt.day == dateVar.date.dt.days_in_month) | \
                     (dateVar.date.dt.day == 15)) * 1

In [24]:
dateVar['month'] = dateVar.date.dt.month

In [25]:
dateVar['monthSinceT0'] = (dateVar.date - pd.datetime(2013,1,1))/np.timedelta64(1, 'M')

In [26]:
dateVar['earthquake'] = (dateVar.date > '2016-04-16') & (dateVar.date <= '2016-04-24')

In [27]:
dateVar.columns.values

array(['date', 'dayOfWeek', 'payDay', 'month', 'monthSinceT0', 'earthquake'], dtype=object)

In [28]:
dateVar[['dayOfWeek', 'payDay', 'month', 'monthSinceT0','earthquake']] = \
    dateVar[['dayOfWeek', 'payDay', 'month', 'monthSinceT0','earthquake']].astype('int8')

In [29]:
oil = pd.read_csv('oil.csv',parse_dates=['date'],infer_datetime_format=True)

In [30]:
dateVar = pd.merge(dateVar,oil,'left').fillna(method='bfill')